In [19]:
!bash ./docker_run.sh python train_DB_StyleGAN2.py --cfg="auto" --snap=20  --data="DATASETS/CrossEyed_256/train/images" --resume="ffhq256" --gpus=1 --mirror=1 --outdir="IJCB_EXPERIMENTS/DB_SG2/experiments_CrossEyed_NIR_RGB_256_NEW_MAY"



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 20,
  "network_snapshot_ticks": 20,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training_scripts_DB_SG2.dataset.ImageFolderDataset",
    "path": "DATASETS/CrossEyed_256/train/images",
    "use_labels": false,
    "max_size": 1728,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training_scripts_DB_SG2.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training_scripts_DB_SG2.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4


# Train and Use Style Interpreter

## 1. Generate Training Images with DB-SG2 

In [22]:
path = "CrossEyed_NIR_RGB"

exp = "IJCB_EXPERIMENTS/interpreter/" + path
sv_path = "IJCB_EXPERIMENTS/interpreter/" + path


In [34]:
exp_generate = exp + "/generate.json" 

# GENERATE training images
!bash docker_run.sh python make_training_data_DB_SG2.py --exp=$exp_generate --sv_path=$sv_path



Opt {'exp_dir': 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB', 'batch_size': 64, 'category': 'eyes_256', 'debug': False, 'dim': [256, 256, 4992], 'deeplab_res': 256, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 8, 'stylegan_ver': '2', 'annotation_data_from_w': False, 'annotation_mask_path': '', 'testing_path': '', 'average_latent': '', 'annotation_image_latent_path': '', 'stylegan_checkpoint': 'IJCB_EXPERIMENTS/checkpoints/IJCB_2022_checkpoints/CrossEyed_NIR_RGB_002080.pkl', 'model_num': 1, 'upsample_mode': 'bilinear', 'upsample_threshold': '8', 'expand_to_dimensions': 14}
cp: 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB/generate.json' and 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB/generate.json' are the same file
Preparing Stylegan{'exp_dir': 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB', 'batch_size': 64, 'category': 'eyes_256', 'debug': False, 'dim': [256, 256, 4992], 'deeplab_res': 256, 'number_class': 4, 'testing_data_number_class': 4, 'max_trai

## Intermediate Steps

### 1.1 Annotate the Generated Images as written in the instructions
### 1.2 Rename avg_latent... to back_avg_latent ... 
### 1.3 run preprocess script (or use your own preprocessing to obtain the required files)

## 2. Train Style Interpreter on Annotated Data 

First create generate and train json files based on the templates

In [42]:
exp_interpreter = exp + "/train_datagan.json" 
!bash docker_run.sh python train_interpreter_DB_SG2.py --exp $exp_interpreter

Device: cuda:0
Opt {'exp_dir': 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB', 'batch_size': 64, 'category': 'eyes_256', 'debug': False, 'dim': [256, 256, 4992], 'deeplab_res': 256, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 8, 'stylegan_ver': '2', 'annotation_data_from_w': False, 'annotation_mask_path': 'eyes_processed', 'testing_path': '', 'average_latent': 'avg_latent_stylegan1.npy', 'annotation_image_latent_path': 'latent_stylegan1.npy', 'stylegan_checkpoint': 'IJCB_EXPERIMENTS/checkpoints/IJCB_2022_checkpoints/CrossEyed_NIR_RGB_002080.pkl', 'model_num': 10, 'upsample_mode': 'bilinear', 'upsample_threshold': '8'}
cp: 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB/train_datagan.json' and 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB/train_datagan.json' are the same file
Prepare data
-- Prepare stylegan
Eyes category
---- Resolution: 256  Layers: 7
---- Get avg latent
---- Latent to torch
AVG latent torch.Size([14, 512])
----  Build Generator
Resuming fro

## 3. Generate Data (RGB, NIR, MASK)

In [45]:
resume_model = "IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB"
num_samples = 100

exp_interpreter = exp + "/train_datagan.json" 
!bash docker_run.sh python train_interpreter_DB_SG2.py --generate_data True --exp=$exp_interpreter  --resume $resume_model  --num_sample=$num_samples


Device: cuda:0
Opt {'exp_dir': 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB', 'batch_size': 64, 'category': 'eyes_256', 'debug': False, 'dim': [256, 256, 4992], 'deeplab_res': 256, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 8, 'stylegan_ver': '2', 'annotation_data_from_w': False, 'annotation_mask_path': 'eyes_processed', 'testing_path': '', 'average_latent': 'avg_latent_stylegan1.npy', 'annotation_image_latent_path': 'latent_stylegan1.npy', 'stylegan_checkpoint': 'IJCB_EXPERIMENTS/checkpoints/IJCB_2022_checkpoints/CrossEyed_NIR_RGB_002080.pkl', 'model_num': 10, 'upsample_mode': 'bilinear', 'upsample_threshold': '8'}
cp: 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB/train_datagan.json' and 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB/train_datagan.json' are the same file
Generate data
Eyes category
---- Resolution: 256  Layers: 7
---- Get avg latent
---- Latent to torch
AVG latent torch.Size([14, 512])
----  Build Generator
Resuming from "IJCB_EXPERIMENTS